In [14]:
import re
from collections import defaultdict

import numpy as np
from numpy.dtypes import StringDType
from scipy.sparse import csr_array

fn = "data/amazon_reviews_2M.csv"
fn = "data/all.csv"
word_filter_re = re.compile(r"[a-zA-Z]+")
line_split_re = re.compile(r"^(.*),(\d+)$", re.DOTALL)

words_to_counts = defaultdict(lambda: defaultdict(lambda: 0))
labels = defaultdict(lambda: 0)
max_label = 0
with open(fn, "r") as f:
    for line in f:
        text, label = line_split_re.findall(line)[0]
        label = int(label)
        max_label = max(max_label, label)
        text = np.unique(np.fromiter((str.lower(word) for word in word_filter_re.findall(text)), dtype=StringDType()))
        labels[label] += 1
        for word in text:
            words_to_counts[word][label] += 1


def get_sum(d):
    res = 0
    for v in d.values():
        res += v
    return res > 1


num_labels = np.array(max_label + 1, dtype=np.int32)

counts = np.fromiter((get_sum(v) for v in words_to_counts.values()), dtype=np.int8)
words = np.fromiter(words_to_counts.keys(), dtype=StringDType())
words = np.array([*words_to_counts.keys()], dtype=StringDType())
print(words)
# mpi --------------------------------------------------------------------------------

ind = words.argsort()
words = words[ind]
words_orig = words.copy()
counts = counts[ind]

# final ---------------------------------------------------------

words = words[np.nonzero(counts)]

label_counts = np.zeros(num_labels, dtype=np.int32)
for label, count in labels.items():
    label_counts[label] = count

res = np.zeros((num_labels, words.shape[0]))

word_locs = np.isin(words, words_orig).nonzero()[0]

print(word_locs)
print(words)
print(words[word_locs])
for loc, word in zip(word_locs, words[word_locs]):
    print(f"{loc} {word}")
    for label, count in words_to_counts[word].items():
        res[label][loc] = count


res_shape = res.shape
res = csr_array(res)

data_in = np.zeros_like(res).flatten()
indices_in = np.zeros_like(data_in)
indptr_in = np.zeros_like(res.indptr)

bufs_in = [data_in, indices_in, indptr_in]
res = res.astype(np.double).toarray()

['not' 'size' 'to' 'true' 'a' 'addictingly' 'amazing' 'an' 'and' 'anymore'
 'are' 'book' 'buy' 'can' 'delicious' 'don' 'eat' 'flavor' 'for' 'get'
 'gets' 'give' 'good' 'i' 'in' 'let' 'like' 'me' 'mean' 'my' 'of' 'one'
 'pack' 'peanuts' 'purchased' 'sitting' 'smoked' 'so' 'stars' 't' 'the'
 'them' 'they' 'think' 'too' 'try' 'unfortunately' 'whole' 'willpower'
 'acting' 'br' 'episode' 'every' 'got' 'great' 'historical' 'it' 'mystery'
 'new' 'right' 'science' 'touch' 'tv' 'with' 'writing' 'biting' 'blow'
 'but' 'dark' 'deception' 'ending' 'even' 'figured' 'full' 'going' 'had'
 'happen' 'haven' 'if' 'many' 'mind' 'moments' 'nail' 'only' 'out'
 'promise' 'revenge' 's' 'secrets' 'simply' 'tense' 'this' 'turns'
 'twists' 've' 'was' 'what' 'will' 'wow' 'you' 'your']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]
['a' 'and' 'book' 'good' 'i' 'in' 'it' 'not' 'of' 'so' 't' 'the' 'they'
 'think' 'to']
['a' 'and' 'book' 'good' 'i' 'in' 'it' 'not' 'of' 'so' 't' 'the' 'they'
 'think' 'to']
0 a
1 and


In [15]:
label_counts = label_counts.astype(np.double)[:, np.newaxis]
label_zeros = label_counts == 0
label_counts[label_zeros] = 1.0

res /= label_counts

label_counts[label_zeros] = 0.0
label_sum = label_counts.sum()
label_counts /= label_sum

In [16]:
res = np.concatenate((res, label_counts), axis=1)

with open("out/all.out", "w") as f:
    f.write(','.join(words))
    f.write('\n')
    np.savetxt(f, res, delimiter=",")

print(res[13])

[1.   1.   0.   1.   0.   1.   1.   0.   1.   0.   0.   0.   1.   0.
 0.   0.25]
